In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, coalesce, lit 
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

# Инициализация SparkSession 
spark = SparkSession.builder \
    .appName("COVID19_ML_Model") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY") \
    .getOrCreate()

optimized_parquet_path = "hdfs:///covid_dataset/metadata_optimized/"

# Путь в HDFS, где хранятся очищенные и оптимизированные метаданные в формате Parquet
df_ml = spark.read.parquet(optimized_parquet_path)

print("Данные для ML загружены из оптимизированного Parquet.")
df_ml.printSchema()

# Целевая переменная: is_covid (0 или 1)
# Признаки: sex, age_group, view, modality, RT_PCR_positive, survival, temperature, pO2_saturation, leukocyte_count, neutrophil_count, lymphocyte_count

# Список категориальных признаков, которые нужно индексировать и One-Hot-кодировать
categorical_features = [
    "sex",
    "age_group",
    "view",
    "modality",
    "RT_PCR_positive",
    "survival"
]

# Список числовых признаков
numeric_features = [
    "age_numeric",
    "temperature",
    "pO2_saturation",
    "leukocyte_count",
    "neutrophil_count",
    "lymphocyte_count"
]

# Валидация и заполнение пропусков в числовых признаках
from pyspark.sql.functions import mean as _mean
for nf in numeric_features:
    if nf in df_ml.columns:
        if df_ml.filter(col(nf).isNull()).count() > 0:
            # Используем average для заполнения
            avg_val = df_ml.select(_mean(col(nf))).collect()[0][0]
            if avg_val is not None:
                df_ml = df_ml.withColumn(nf, col(nf).cast("double"))
                df_ml = df_ml.na.fill(avg_val, subset=[nf]) 
                print(f"Заполнены пропуски в {nf} средним {avg_val:.2f}")
            else:
                print(f"Предупреждение: Колонка {nf} содержит только NULL значения, невозможно заполнить средним.")
                df_ml = df_ml.withColumn(nf, lit(0.0).cast("double")) # Заполняем 0.0 если все NULL
    else:
        print(f"Предупреждение: Числовая колонка '{nf}' не найдена в DataFrame. Пропускаем.")
        numeric_features.remove(nf) # Удаляем несуществующие колонки из списка

# Создаем стадии Pipeline для StringIndexer и OneHotEncoderEstimator
indexers = [
    StringIndexer(inputCol=feature, outputCol=feature + "_indexed", handleInvalid="keep")
    for feature in categorical_features if feature in df_ml.columns
]

encoders = [
    OneHotEncoder(inputCol=feature + "_indexed", outputCol=feature + "_encoded")
    for feature in categorical_features if feature in df_ml.columns # Аналогично
]

assembler_inputs = [f.getOutputCol() for f in encoders] + numeric_features

if not assembler_inputs:
    raise ValueError("Не найдено действительных признаков для VectorAssembler после всех преобразований. Проверьте списки признаков и схему DataFrame.")

vector_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Модель классификации: Логистическая регрессия
lr = LogisticRegression(featuresCol="features", labelCol="is_covid", maxIter=10)

# Создание Pipeline
pipeline = Pipeline(stages=indexers + encoders + [vector_assembler, lr])

print("Начинаем разделение данных на обучающую и тестовую выборки...")
# Разделение данных на обучающую 80% и тестовую 20% 
(training_data, test_data) = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"Обучающая выборка: {training_data.count()} строк")
print(f"Тестовая выборка: {test_data.count()} строк")

print("Начинаем обучение модели...")
# Обучение модели
model = pipeline.fit(training_data)
print("Обучение модели завершено.")

# Предсказания на тестовой выборке
print("Выполняем предсказания на тестовой выборке...")
predictions = model.transform(test_data)
predictions.select("patientid", "is_covid", "prediction", "probability").show(10, truncate=False)

# Оценка модели
print("Оцениваем производительность модели...")
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="is_covid", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)
print(f"Area Under ROC (AUC) на тестовой выборке: {auc:.4f}")

try:
    lr_model = model.stages[-1]
    print(f"\nКоличество признаков в модели: {lr_model.numFeatures}")
    if lr_model.numFeatures > 0:
        print(f"Коэффициенты модели логистической регрессии: {lr_model.coefficientMatrix.toArray()}")
    print(f"Пересечение (Intercept): {lr_model.intercept}")
except Exception as e:
    print(f"\nНе удалось получить коэффициенты логистической регрессии: {e}")

spark.stop()
print("SparkSession остановлена. Обучение и оценка ML-модели завершены.")

Данные для ML загружены из оптимизированного Parquet.
root
 |-- patientid: string (nullable = true)
 |-- offset: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- finding: string (nullable = true)
 |-- RT_PCR_positive: string (nullable = true)
 |-- survival: string (nullable = true)
 |-- intubated: string (nullable = true)
 |-- intubation_present: string (nullable = true)
 |-- went_icu: string (nullable = true)
 |-- in_icu: string (nullable = true)
 |-- needed_supplemental_O2: string (nullable = true)
 |-- extubated: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- pO2_saturation: double (nullable = true)
 |-- leukocyte_count: double (nullable = true)
 |-- neutrophil_count: double (nullable = true)
 |-- lymphocyte_count: double (nullable = true)
 |-- view: string (nullable = true)
 |-- modality: string (nullable = true)
 |-- date: string (nullable = true)
 |-- location: string (nullable = true)
 |-- folder: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- url: string (nullable = true)
 |-- license: string (nullable = true)
 |-- clinical_notes: string (nullable = true)
 |-- other_notes: string (nullable = true)
 |-- finding_unified: string (nullable = true)
 |-- date_parsed: date (nullable = true)
 |-- day: integer (nullable = true)
 |-- is_covid: integer (nullable = true)
 |-- age_numeric: integer (nullable = true)
 |-- age_group: string (nullable = true)
 |-- hdfs_image_path: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

Заполнены пропуски в leukocyte_count средним 5.02
Заполнены пропуски в neutrophil_count средним 5.31
Заполнены пропуски в lymphocyte_count средним 4.64
Начинаем разделение данных на обучающую и тестовую выборки...
Обучающая выборка: 797 строк
Тестовая выборка: 153 строк
Начинаем обучение модели...
Обучение модели завершено.
Выполняем предсказания на тестовой выборке...
+---------+--------+----------+------------------------------------------+
|patientid|is_covid|prediction|probability                               |
+---------+--------+----------+------------------------------------------+
|112      |1       |0.0       |[0.7196708899989618,0.2803291100010382]   |
|114      |1       |0.0       |[0.6008662980254003,0.39913370197459974]  |
|115      |1       |0.0       |[0.6008662980254003,0.39913370197459974]  |
|117      |1       |0.0       |[0.5561334789784026,0.4438665210215974]   |
|12       |1       |1.0       |[0.16294338593720847,0.8370566140627915]  |
|123      |1       |1.0       |[0.1152293800054515,0.8847706199945485]   |
|129      |1       |1.0       |[0.10795132272644777,0.8920486772735522]  |
|13       |1       |1.0       |[0.18540709049829673,0.8145929095017033]  |
|134      |1       |1.0       |[0.0028766742190906567,0.9971233257809093]|
|135      |1       |1.0       |[0.0014749520401481683,0.9985250479598519]|
+---------+--------+----------+------------------------------------------+
only showing top 10 rows

Оцениваем производительность модели...
Area Under ROC (AUC) на тестовой выборке: 0.8835

Количество признаков в модели: 24
Коэффициенты модели логистической регрессии: [[ 0.15569132 -0.15569132  0.03247601 -0.03247601 -0.12611848  0.47729336
   0.05383687 -1.53134928  0.49917676  2.12085453 -4.55020856 -0.85601475
   0.85601475 -1.95999543  1.95999543  0.38295352 -0.15080019 -0.80309475
   0.01835915 -0.07047965 -0.10493309  0.64478136  0.01838265 -0.02836752]]
Пересечение (Intercept): 10.050194758470186
SparkSession остановлена. Обучение и оценка ML-модели завершены.